# Unfitted FEM discretizations

We want to solve a geometrically unfitted model problem for a *stationary* domain.

We use a level set description (cf. [basics.ipynb](basics.ipynb)): 

$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

and use a piecewise linear approximation as *a starting point* in the discretization (cf. [intlset.ipynb](intlset.ipynb) for a discussion of geometry approximations). 


We first import the related packages: 


In [ ]:
import netgen.gui
# the constant pi
from math import pi
# ngsolve stuff
from ngsolve import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import * 


![title](graphics/bubble_coarse.png)

## Interface problem 

We want to solve a problem of the form: 

$$
\left\{
\begin{aligned}
          - \nabla \cdot (\alpha_{\pm} \nabla u) = & \, f 
          & & \text{in}~~ \Omega_{\pm}, 
          \\
          [\![u]\!] = & \, 0 
          &  & \text{on}~~ \Gamma, 
          \\
          [\![-\alpha \nabla u \cdot \mathbf{n}]\!]   = & \, 0 
          &  & \text{on}~~ \Gamma,
          \\
          u = & \, u_D
          &  & \text{on}~~ \partial \Omega.
        \end{aligned} \right.
$$


In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.4, quad_dominated=False))

levelset = (sqrt(x*x+y*y) - 1.0)
Draw(levelset,mesh,"levelset")

lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
Draw(lsetp1,mesh,"lsetp1")

visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

### Cut FE spaces

For the discretization we use standard background FESpaces restricted to the subdomains: 

$$
V_h^\Gamma \quad = \qquad V_h |_{\Omega_-^{lin}} \quad \oplus \quad V_h |_{\Omega_+^{lin}}
$$

| composed | inner | outer |
|-|-|-|
| ![alt](graphics/SpaceBoth.png) | ![alt](graphics/SpaceInner.png) | ![alt](graphics/SpaceOuter.png) |

In NGSolve we simply take the product space $V_h \times V_h$ but mark the irrelevant dofs using the CutInfo-class:


In [ ]:
Vh = H1(mesh, order=2, dirichlet=[1,2,3,4])
VhG = FESpace([Vh,Vh])

ci = CutInfo(mesh, lsetp1)
freedofs = VhG.FreeDofs()
freedofs &= CompoundBitArray([GetDofsOfElements(Vh,ci.GetElementsOfType(HASNEG)),
                              GetDofsOfElements(Vh,ci.GetElementsOfType(HASPOS))])

gfu = GridFunction(VhG)

### Let us visualize active dofs:
* active dofs of first space are set to -1
* active dofs of second space are set to 1
* inactive dofs are 0

In [ ]:
gfu.components[0].Set(CoefficientFunction(-1))
gfu.components[1].Set(CoefficientFunction(1))
for i, val in enumerate(freedofs):
    if not val:
        gfu.vec[i] = 0.0

In [ ]:
Draw(gfu.components[0],mesh,"background_uneg")

In [ ]:
Draw(gfu.components[1],mesh,"background_upos")

Only the parts which are in the corresponding subdomain are relevant. The solution $u$ is: 

$$
 u = \left\{ \begin{array}{cc} u_- & \text{ if } {\phi}_h^{lin} < 0, \\ u_+ & \text{ if } {\phi}_h^{lin} \geq 0. \end{array} \right.
$$


In [ ]:
# uh (only for visualization purpose)
uh = IfPos(lsetp1,gfu.components[1],gfu.components[0])
Draw(uh,mesh,"u",sd=5)

### Improvement: use `Compress` to reduce unused dofs

In [ ]:
Vh = H1(mesh, order=2, dirichlet=[1,2,3,4])
ci = CutInfo(mesh, lsetp1)
VhG = FESpace([Compress(Vh,GetDofsOfElements(Vh,ci.GetElementsOfType(cdt))) for cdt in [HASNEG,HASPOS]])

freedofs = VhG.FreeDofs()
gfu = GridFunction(VhG)
uh = IfPos(lsetp1,gfu.components[1],gfu.components[0])

Draw(uh,mesh,"u",sd=5)
print(Vh.ndof, VhG.components[0].ndof, VhG.components[1].ndof)

## Nitsche discretization

For the discretization of the interface problem we consider the Nitsche formulation:
\begin{align*}
         \sum_{i \in \{+,-\}} & \left( \alpha_i \nabla u \nabla v \right)_{\Omega_i} + \left( \{\!\!\{ - \alpha \nabla u \cdot n \}\!\!\}, [\![v]\!] \right)_\Gamma + \left( [\![u]\!],\{\!\!\{ - \alpha \nabla v \cdot n \}\!\!\} \right)_\Gamma + \left( \frac{\bar{\alpha} \lambda}{h}  [\![u]\!] , [\![v]\!] \right)_{\Gamma} \\
         = \left( f,v \right)_{\Omega}
\end{align*}

for all $v \in V_h^\Gamma$.

For this formulation we require:
* a suitably defined average operator $\{ \cdot \} = \kappa_+ (\cdot)|_{\Omega_{+}} + \kappa_- (\cdot)|_{\Omega_{-}}$
* a suitable definition of the normal direction
* numerical integration on $\Omega_{+}^{lin}$, $\Omega_{-}^{lin}$ and $\Gamma^{lin}$

### Cut ratio field

For the average we use the "Hansbo"-choice: 

$$
\kappa_- = \frac{|T \cap \Omega_-^{lin}|}{|T|} \qquad 
\kappa_+ = 1 - \kappa_- 
$$

This "cut ratio" field is provided by the CutInfo class: 


In [ ]:
kappaminus = CutRatioGF(ci)
kappa = (kappaminus, 1-kappaminus)
Draw(kappaminus,mesh,"kappa")

### Normal direction

The normal direction is obtained from the (interpolated) level set function:

$$
  n^{lin} = \frac{\nabla \phi_h^{lin}}{\Vert \nabla \phi_h^{lin} \Vert}
$$


In [ ]:
n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
Draw(n,mesh,"normal")

### Averages and jumps

Based on the background finite elements we can now define the averages and jumps: 


In [ ]:
h = specialcf.mesh_size

alpha = [1.0,20.0]

# Nitsche stabilization parameter:
stab = 20*(alpha[1]+alpha[0])/h

# expressions of test and trial functions (u and v are tuples):
u,v = VhG.TnT()

average_flux_u = sum([- kappa[i] * alpha[i] * grad(u[i]) * n for i in [0,1]])
average_flux_v = sum([- kappa[i] * alpha[i] * grad(v[i]) * n for i in [0,1]])

jump_u = u[0] - u[1]
jump_v = v[0] - v[1]

### Integrals

To integrate only on the subdomains or the interface with a SymbolicBFI you have to add a "levelset_domain" argument which is a dictionary, cf. [intlset.ipynb](intlset.ipynb): 


In [ ]:
lset_neg = { "levelset" : lsetp1, "domain_type" : NEG}
lset_pos = { "levelset" : lsetp1, "domain_type" : POS}
lset_if  = { "levelset" : lsetp1, "domain_type" : IF }

We first integrate over the subdomains: 

$$
\int_{\Omega_-} \alpha_- \nabla u \nabla v \, d\omega 
$$

$$
\int_{\Omega_+} \alpha_+ \nabla u \nabla v \, d\omega 
$$


In [ ]:
bfi_neg = SymbolicBFI(levelset_domain = lset_neg, form = alpha[0] * grad(u[0]) * grad(v[0]))
bfi_pos = SymbolicBFI(levelset_domain = lset_pos, form = alpha[1] * grad(u[1]) * grad(v[1]))

We then integrate over the interface: 

$$
        \int_{\Gamma} \{\!\!\{ - \alpha \nabla u \cdot \mathbf{n} \}\!\!\} [\![v]\!] \, d\gamma 
         + \int_{\Gamma} \{\!\!\{ - \alpha \nabla v \cdot \mathbf{n} \}\!\!\} [\![u]\!] \, d\gamma 
         + \int_{\Gamma} \frac{\lambda}{h} \bar{\alpha} [\![u]\!] [\![v]\!] \, d\gamma
$$


In [ ]:
bfi_if  = SymbolicBFI(levelset_domain = lset_if , form =       average_flux_u * jump_v
                                                             + average_flux_v * jump_u
                                                             +  stab * jump_u * jump_v )

We sum them up to get the bilinear form in full: 


In [ ]:
a = BilinearForm(VhG, symmetric = True, flags = { })
for bfi in [bfi_neg,bfi_pos,bfi_if]:
    a += bfi

Finally, we integrate over the subdomains to get the linear form: 

$$
f(v) = \int_{\Omega_-} f_- v \, d\omega + 
       \int_{\Omega_+} f_+ v \, d\omega
$$


In [ ]:
coef_f = [1,0]    
lfi_neg = SymbolicLFI(levelset_domain = lset_neg, form = coef_f[0] * v[0])
lfi_pos = SymbolicLFI(levelset_domain = lset_pos, form = coef_f[1] * v[1])

f = LinearForm(VhG)
f += lfi_neg
f += lfi_pos  

### Assembly

(Only) to speed up assembly we can mark the integrators as undefined where they would be zero anyway: 


In [ ]:
bfi_neg.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
lfi_neg.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
bfi_pos.SetDefinedOnElements(ci.GetElementsOfType(HASPOS))
lfi_pos.SetDefinedOnElements(ci.GetElementsOfType(HASPOS))
bfi_if.SetDefinedOnElements(ci.GetElementsOfType(IF))
a.Assemble()
f.Assemble()

We can now solve the problem (recall that freedofs only marks relevant dofs): 


In [ ]:
# homogenization of boundary data and solution of linear system
def SolveLinearSystem():
    rhs = gfu.vec.CreateVector()
    rhs.data = f.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(freedofs) * rhs
    gfu.vec.data += update
SolveLinearSystem()

uh = IfPos(lsetp1,gfu.components[1],gfu.components[0])
Draw(uh,mesh,"u",sd=5)
#Draw(uh,mesh,"gfu")

## Higher order accuracy
In the previous example we used a second order FESpace but only used a second order accurate geometry representation (due to $\phi_h^{lin}$). 

We can improve this by applying a mesh transformation technique, cf. [intlset.ipynb](intlset.ipynb):

![title](graphics/lsetcurv.jpg)

In [ ]:
# for isoparametric mapping
from xfem.lsetcurv import *
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2)
deformation = lsetmeshadap.CalcDeformation(levelset)
Draw(deformation,mesh,"deformation")

mesh.SetDeformation(deformation)
a.Assemble()
f.Assemble()
mesh.UnsetDeformation()
SolveLinearSystem()

visoptions.deformation = 1.0
Draw(uh,mesh,"u",sd=5)
Draw(CoefficientFunction((deformation[0],deformation[1],4*uh)),mesh,"graph_of_u",sd=5)

## XFEM spaces

Instead of the CutFEM space 
$$
V_h^\Gamma = V_h |_{\Omega_-^{lin}} \oplus V_h |_{\Omega_+^{lin}}
$$
we can use the (same) space with an XFEM characterization:
$$
V_h^\Gamma = V_h \oplus V_h^x
$$
with the space $V_h^x$ which adds the necessary enrichments. 

In `ngsxfem` we can also work with this XFEM spaces: 


In [ ]:
Vh = H1(mesh, order=2, dirichlet=[1,2,3,4])
Vhx = XFESpace(Vh,ci)
VhG = FESpace([Vh,Vhx])


| original | after cut |
|-|-|
| ![alt](graphics/xfem1.png) | ![alt](graphics/xfem2.png) |


* The space `Vhx` copies all shape functions from `Vh` on cut (`IF`) elements (and stores a sign (`NEG`/`POS`))
* The sign determines on which domain the shape function should be supported (and where not)
* Advantage: every dof is an active dof (i.e. no dummy dofs)
* Need to express $u_+$ and $u_-$ in terms of $u_h^{std}$ and $u_h^x$:
  * $u_- = u_h^{std} + $ `neg`($u_h^x$) and $u_+ = u_h^{std} + $ `pos`($u_h^x$)
* `neg` and `pos` filter out the right shape functions of `Vhx`

In [ ]:
### express xfem shape functions as cutFEM shape functions:
(u_std,u_x), (v_std, v_x) = VhG.TnT()

u = [u_std + op(u_x) for op in [neg,pos]]
v = [v_std + op(v_x) for op in [neg,pos]]

## Similar examples and extensions (python demo files)
In the source directory (or on http://www.github.com/ngsxfem/ngsxfem ) you can find (in the `py_tutorial` directory):
* `cutfem.py` : similar to the low order case in this notebook
* `nxfem.py` : same discretization but with a XFEM-type setup of the spaces
* `nxfem_higher_order.py` : same discretization as in the section "higher order accuracy" but with XFEM-type spaces
* `fictdom_ghostpen.py` : Fictitious domain problem (one domain only) with a ghost penalty stabilization
* `fictdom_dg_ghostpen.py` : Fictitious domain problem in DG formulation with a ghost penalty stabilization
* `stokesxfem.py` : Stokes interface problem with XFEM + Nitsche
* `tracefem3D.py` or [tracefem_scalar.ipynb](tracefem_scalar.ipynb) : Trace Finite element discretization (PDE on the interface only)